In [13]:
import sys
!{sys.executable} -m pip install -U scikit-learn==0.20.3 --user

import scipy.io as spio
from scipy import signal
from scipy.stats import kurtosis, skew
from statsmodels.robust.scale import mad
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.dummy import DummyRegressor
from scipy.interpolate import UnivariateSpline, interp1d
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, RandomForestRegressor, ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import accuracy_score, r2_score
from scipy.signal import savgol_filter, butter, lfilter, welch
from joblib import dump, load


!{sys.executable} -m pip install peakutils
import peakutils as pk
from peakutils.plot import plot as pplot

!{sys.executable} -m pip install hyperopt --user
!{sys.executable} -m pip install --upgrade git+https://github.com/hyperopt/hyperopt-sklearn.git
from hpsklearn import HyperoptEstimator, knn, ada_boost, gradient_boosting, sgd, multinomial_nb, gaussian_nb, extra_trees, random_forest, any_classifier, svc, any_regressor, any_preprocessing, random_forest_regression, extra_trees_regression, gradient_boosting_regression, sgd_regression
from hyperopt import tpe

Requirement already up-to-date: scikit-learn==0.20.3 in c:\users\kamil\appdata\roaming\python\python37\site-packages (0.20.3)
  Cloning https://github.com/hyperopt/hyperopt-sklearn.git to c:\users\kamil\appdata\local\temp\pip-req-build-ifjct2tp
  Stored in directory: C:\Users\Kamil\AppData\Local\Temp\pip-ephem-wheel-cache-0bmz5xhn\wheels\28\93\20\67dca95c2aaa13466b4900ba79a7bab66022e50ce44f8a438d
Successfully built hpsklearn
  Found existing installation: hpsklearn 0.0.3
    Uninstalling hpsklearn-0.0.3:
      Successfully uninstalled hpsklearn-0.0.3


  Running command git clone -q https://github.com/hyperopt/hyperopt-sklearn.git 'C:\Users\Kamil\AppData\Local\Temp\pip-req-build-ifjct2tp'


In [2]:
import sklearn
print(sklearn.__version__)

0.20.3


## Preparing functions for row extraction

In [3]:
max_hr = 135
alpha = 0.7
print_plots = False
min_hr = 50

# based on https://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y  
  
def extract_frequent_domain_features(rr):
  rr_in_ms = rr*1000
  nni_tmstp = np.cumsum(rr)
  nni_tmstp = nni_tmstp - nni_tmstp[0]
  funct = interp1d(x=nni_tmstp, y=rr_in_ms, kind='cubic')
  timestamps_interpolation = np.arange(0, nni_tmstp[-1], 1 / float(4))
  nni_interpolation = funct(timestamps_interpolation)
  nni_normalized = nni_interpolation - np.mean(nni_interpolation)
  freq, psd = signal.welch(x=nni_normalized, fs=4)
#   print(len(psd))
  lf_indexes = np.logical_and(freq >= 0.04, freq < 0.15)
  hf_indexes = np.logical_and(freq >= 0.15, freq < 0.4)
  lf = np.trapz(y=psd[lf_indexes], x=freq[lf_indexes])
  hf = np.trapz(y=psd[hf_indexes], x=freq[hf_indexes])
  return lf, hf, psd
  
  
def calculate_hrv_attrs(rr):
  rmssd = 0
  diffs = []
  nn20 = 0
  nn50 = 0
  for i in range(1,len(rr)-1):
    diff = abs(rr[i+1] - rr[i])
    if(diff > 0.05):
      nn50+=1
    if(diff > 0.02):
      nn20+=1
    diffs.append(diff)
    rmssd += (diff)**2
  rmssd = math.sqrt(rmssd/(len(rr)-1))
  sdsd = np.std(diffs)
  sd1 = np.sqrt(np.std(diffs) ** 2 * 0.5)
  sd2 = np.sqrt(2 * np.std(rr) ** 2 - 0.5 * np.std(diffs) ** 2)
  
  lf, hf, per = extract_frequent_domain_features(rr)
  
  return {
    'rmssd': rmssd,
    'pnn20': nn20/len(rr),
    'pnn50': nn50/len(rr),
    'sdsd': sdsd,
    'sdnn': np.std(rr),
    'sd1': sd1,
    'sd2': sd2,
    'sd2_sd1_ratio': sd2/sd1,
    'lf': lf,
    'hf': hf,
    'lfhf': lf/hf,
    'per': per
  }

def extract_row(rr_data, data_frequency, valence, arousal):
  time = int(len(rr_data)/data_frequency)
  min_dist = 60*data_frequency/max_hr
  indexes = pk.indexes(rr_data, min_dist=min_dist)
  rr = np.diff(indexes)/data_frequency
  if (len(rr) < 10):
    x = np.linspace(0,len(rr_data),len(rr_data))
    plt.figure()
    pplot(x,rr_data,indexes)
  rr_median = np.median(rr)
  med_filt = np.where(np.logical_and(rr >= alpha*rr_median, rr <= (2-alpha) * rr_median))
  med_filt = np.where(np.logical_and(rr >= alpha*rr_median, rr <= (2-alpha) * rr_median))
  filtered_indices = indexes[med_filt]
  filtered_rr = rr[med_filt]
  hr = 60/filtered_rr
  hrv_attrs = calculate_hrv_attrs(filtered_rr)
  mean_hr = np.mean(hr)
  mean_rr = np.mean(filtered_rr)
  std_hr = np.std(hr)
  std_rr = hrv_attrs['sdnn']
  
  hr_above_mean_plus_std = hr[np.where(hr > mean_hr + std_hr)]
  hr_below_mean_minus_std = hr[np.where(hr < mean_hr - std_hr)]
  rr_above_mean_plus_std = filtered_rr[np.where(filtered_rr > mean_rr + std_rr)]
  rr_below_mean_minus_std = filtered_rr[np.where(filtered_rr < mean_rr - std_rr)]
  
  return {
#     'filtered_indexes': filtered_indices,
#     'rr': filtered_rr,
#     'per': hrv_attrs['per'],
    'mean_hr': mean_hr,
    'min_hr': hr.min(),
    'max_hr': hr.max(),
    'std_hr': std_hr,
    'kurtosis_hr': kurtosis(hr),
    'skewness_hr': skew(hr),
    'hr_above_mean_plus_std': len(hr_above_mean_plus_std)/len(hr),
    'hr_below_mean_minus_std': len(hr_below_mean_minus_std)/len(hr),
    'mad_rr': mad(filtered_rr),
    'mean_rr': mean_rr,
    'kurtosis_rr': kurtosis(filtered_rr),
    'skewness_rr': skew(filtered_rr),
    'rr_above_mean_plus_std': len(rr_above_mean_plus_std)/len(filtered_rr),
    'rr_below_mean_minus_std': len(rr_below_mean_minus_std)/len(filtered_rr),
    'sdnn': hrv_attrs['sdnn'],
    'rmssd': hrv_attrs['rmssd'],
    'sdsd': hrv_attrs['sdsd'],
    'pnn20': hrv_attrs['pnn20'],
    'pnn50': hrv_attrs['pnn50'],
    'sd1': hrv_attrs['sd1'],
    'sd2': hrv_attrs['sd2'],
    'sd2_sd1_ratio': hrv_attrs['sd2_sd1_ratio'],
    'lf': hrv_attrs['lf'],
    'hf': hrv_attrs['hf'],
    'lfhf': hrv_attrs['lfhf'],
    'valence': valence,
    'arousal': arousal
  }

## ASCERTAIN dataset preprocessing and feature extraction

In [14]:
ascertain_subjects = 58
ascertain_videos = 36
ascertain_frequency = 256
ascertain_lowcut = 8
ascertain_highcut = 20
ascertain_order = 8
ascertain_last_n_seconds_start = 50
  
ecg_features = spio.loadmat('D:\Physiological Data\ASCERTAIN\ASCERTAIN_Features\Dt_ECGFeatures.mat')
movie_orders = spio.loadmat('D:\Physiological Data\ASCERTAIN\ASCERTAIN_Features\Dt_Order_Movie.mat')['PermutationList']
data_quality = np.genfromtxt('D:\Physiological Data\ASCERTAIN\ASCERTAIN_Features\Data_Quality_Evaluation.csv', delimiter=',')
self_reports = spio.loadmat('D:\Physiological Data\ASCERTAIN\ASCERTAIN_Features\Dt_SelfReports.mat')

arousal = self_reports['Ratings'][0]
valence = self_reports['Ratings'][1]
arousal = (9-1)*(arousal-arousal.min())/(arousal.max()-arousal.min()) + 1
valence = (9-1)*(valence-valence.min())/(valence.max()-valence.min()) + 1

failures = ecg_features['ECGFailures_58'][0]
ascertain_dataset = []

for sub in range(0,ascertain_subjects):
  for video in range(0,ascertain_videos):
    if data_quality[video,sub] > 5 or video in failures[sub]:
#       print(str(sub) + " " + str(perm))
      continue
    perm = movie_orders[sub, video]
    loaded_data = spio.loadmat('D:\Physiological Data\ASCERTAIN\ASCERTAIN_Raw\ECGData\Movie_P'+str(sub+1).zfill(2) + '\ECG_clip' + str(perm) + '.mat')
    ecg_data = pd.DataFrame(loaded_data['Data_ECG'])
    left_arm_index = 4 if len(ecg_data.columns) > 3 else 1
    right_arm_index = 5 if len(ecg_data.columns) > 3 else 2
    left_arm = butter_bandpass_filter(ecg_data[left_arm_index].to_numpy(), ascertain_lowcut, ascertain_highcut, ascertain_frequency, order=ascertain_order)[-1*ascertain_last_n_seconds_start*ascertain_frequency:]
    right_arm = butter_bandpass_filter(ecg_data[right_arm_index].to_numpy(), ascertain_lowcut, ascertain_highcut, ascertain_frequency, order=ascertain_order)[-1*ascertain_last_n_seconds_start*ascertain_frequency:]
    val = valence[sub,video]
    ar = arousal[sub,video]
    try:
      ascertain_dataset.append(extract_row(left_arm, ascertain_frequency, val, ar))
      ascertain_dataset.append(extract_row(right_arm, ascertain_frequency, val, ar))
    except:
      print(str(sub) + " " + str(video))
len(ascertain_dataset)    

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 188, using nperseg = 188
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 188, using nperseg = 188
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 194, using nperseg = 194
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 196, using nperseg = 196
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 196, using nperseg = 196
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 193, using nperseg = 193
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 193, using nperseg = 193
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 134, using nperseg = 134
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 72, using nperseg = 72
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 164, using nperseg = 164
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 69, using nperseg = 69
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\s

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 169, using nperseg = 169
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 81, using nperseg = 81
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 150, using nperseg = 150
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 113, using nperseg = 113
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 168, using nperseg = 168
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 148, using nperseg = 148
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 146, using nperseg = 146
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 152, using nperseg = 152
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 155, using nperseg = 155
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 119, using nperseg = 119
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

...

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 98, using nperseg = 98
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 78, using nperseg = 78
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 120, using nperseg = 120
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 101, using nperseg = 101
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\sci

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 145, using nperseg = 145
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 84, using nperseg = 84
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 152, using nperseg = 152
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 155, using nperseg = 155
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 132, using nperseg = 132
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 139, using nperseg = 139
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 176, using nperseg = 176
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 175, using nperseg = 175
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 192, using nperseg = 192
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 192, using nperseg = 192
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 107, using nperseg = 107
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 124, using nperseg = 124
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 172, using nperseg = 172
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 179, using nperseg = 179
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 193, using nperseg = 193
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

3884

## DEAP dataset preprocessing and feature extraction

In [15]:
deap_frequency = 128
deap_lowcut = 8
deap_highcut = 20
deap_order = 8
start_time = 10

def preprocess_features(dataset, video):
  respiration_belt = butter_bandpass_filter(dataset['data'][video][38], deap_lowcut, deap_highcut, deap_frequency, deap_order)
  respiration_belt = respiration_belt[start_time*deap_frequency:]
#   Valence and arousal already in 1-9 range, do not need to scale it
  labels = dataset['labels'][video]
  return extract_row(respiration_belt, deap_frequency, labels[0], labels[1])

deap_dataset = []
for i in range(1,33):
  s01 = spio.loadmat('D:\Physiological Data\DEAP (Database for Emotion Analysis using Physiological Signals)\data_preprocessed_matlab\s' +str(i).zfill(2)  + '.mat')
  for video_number in range(0,len(s01['labels'])):
    deap_dataset.append(preprocess_features(s01, video_number))
len(deap_dataset)

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 133, using nperseg = 133
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 178, using nperseg = 178
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 114, using nperseg = 114
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 170, using nperseg = 170
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 136, using nperseg = 136
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 131, using nperseg = 131
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 145, using nperseg = 145
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 166, using nperseg = 166
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 140, using nperseg = 140
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 180, using nperseg = 180
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 210, using nperseg = 210
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 207, using nperseg = 207
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 203, using nperseg = 203
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 204, using nperseg = 204
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 166, using nperseg = 166
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 177, using nperseg = 177
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 158, using nperseg = 158
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 185, using nperseg = 185
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 165, using nperseg = 165
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 185, using nperseg = 185
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 170, using nperseg = 170
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 198, using nperseg = 198
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 185, using nperseg = 185
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 186, using nperseg = 186
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

...

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 130, using nperseg = 130
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 156, using nperseg = 156
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 150, using nperseg = 150
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 151, using nperseg = 151
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 165, using nperseg = 165
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 193, using nperseg = 193
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 193, using nperseg = 193
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 191, using nperseg = 191
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 192, using nperseg = 192
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 142, using nperseg = 142
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 141, using nperseg = 141
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 151, using nperseg = 151
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 170, using nperseg = 170
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 153, using nperseg = 153
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 142, using nperseg = 142
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 153, using nperseg = 153
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 166, using nperseg = 166
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 166, using nperseg = 166
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 145, using nperseg = 145
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

1280

## DECAF dataset preprocessing and feature extraction

In [16]:
decaf_frequency = 1000
decaf_subjects = 30
decaf_videos = 36
decaf_last_n_seconds_start = 50
decaf_lowcut = 8
decaf_highcut = 20
decaf_order = 4

decaf_labels = spio.loadmat('D:\Physiological Data\DECAF\DECAF-CLS-Features\Dt_SelfAssessments_Movie_Sorted.mat')
decaf_video_order = spio.loadmat('D:\Physiological Data\DECAF\DECAF-CLS-Features\Dt_Order_Movie.mat')['PermutationList']
arousal = decaf_labels['SelfAssessments'][0][0][0]
valence = decaf_labels['SelfAssessments'][0][0][1]
arousal = (9-1)*(arousal-arousal.min())/(arousal.max()-arousal.min()) + 1
valence = (9-1)*(valence-valence.min())/(valence.max()-valence.min()) + 1

decaf_dataset = []
for sub in range(0,decaf_subjects):
  for video in range(0,decaf_videos):
    perm = decaf_video_order[sub,video]
    loaded_data = spio.loadmat('D:\Physiological Data\DECAF\DECAF-PreProcessed\MovieDataSegments-Face-Phys\MovieDataSegments\ECGData\S'+str(sub+1).zfill(2) + '\ECG_clip' + str(perm).zfill(2) + '.mat')
    ecg_data = butter_bandpass_filter(loaded_data['ThsECG'][0], decaf_lowcut, decaf_highcut, decaf_frequency, decaf_order)[-1*decaf_last_n_seconds_start*decaf_frequency:]
    val = valence[sub,video]
    ar = arousal[sub,video]
    decaf_dataset.append(extract_row(ecg_data, decaf_frequency, val, ar))
len(decaf_dataset)

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 148, using nperseg = 148
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 85, using nperseg = 85
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 161, using nperseg = 161
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 150, using nperseg = 150
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 162, using nperseg = 162
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 192, using nperseg = 192
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 143, using nperseg = 143
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 192, using nperseg = 192
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 82, using nperseg = 82
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 190, using nperseg = 190
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 191, using nperseg = 191
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 196, using nperseg = 196
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 196, using nperseg = 196
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 188, using nperseg = 188
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 197, using nperseg = 197
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 182, using nperseg = 182
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 194, using nperseg = 194
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 185, using nperseg = 185
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 178, using nperseg = 178
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 88, using nperseg = 88
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 166, using nperseg = 166
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 197, using nperseg = 197
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 185, using nperseg = 185
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 178, using nperseg = 178
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 164, using nperseg = 164
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

...

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 150, using nperseg = 150
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 118, using nperseg = 118
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 143, using nperseg = 143
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 146, using nperseg = 146
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 198, using nperseg = 198
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 193, using nperseg = 193
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 198, using nperseg = 198
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 197, using nperseg = 197
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 196, using nperseg = 196
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 138, using nperseg = 138
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 152, using nperseg = 152
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 142, using nperseg = 142
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 120, using nperseg = 120
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 159, using nperseg = 159
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 101, using nperseg = 101
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 121, using nperseg = 121
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 197, using nperseg = 197
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 101, using nperseg = 101
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 96, using nperseg = 96
  .format(nperseg, input_length))


1080

## AMIGOS dataset preprocessing and feature extraction

In [17]:
amigos_subjects = 40
amigos_videos = 16 # taking only short videos to have consistency with other datasets
amigos_frequency = 128 # because using preprocessed data
amigos_lowcut = 8
amigos_highcut = 20
amigos_order = 8
amigos_ecg_leftarm = 14
amigos_ecg_rightarm = 15
amigos_last_n_seconds_start = 50

amigos_dataset = []
for sub in range(0,amigos_subjects):
  matfile = spio.loadmat('D:\Physiological Data\AMIGOS (affect, personality and mood research on individuals and groups)\Data Preprocessed\Data_Preprocessed_P' + str(sub+1).zfill(2) + '\Data_Preprocessed_P'+ str(sub+1).zfill(2) + '.mat')
  joined_data = matfile['joined_data'][0]
  labels = matfile['labels_selfassessment'][0]
  for video in range(0,amigos_videos):
    data_as_df = pd.DataFrame(joined_data[video])
    valence = labels[video][0][1]
    arousal = labels[video][0][0]
    leftarm = butter_bandpass_filter(data_as_df[amigos_ecg_leftarm].to_numpy(), amigos_lowcut, amigos_highcut, amigos_frequency, amigos_order)[-1*amigos_last_n_seconds_start*amigos_frequency:]
    rightarm = butter_bandpass_filter(data_as_df[amigos_ecg_rightarm].to_numpy(), amigos_lowcut, amigos_highcut, amigos_frequency, amigos_order)[-1*amigos_last_n_seconds_start*amigos_frequency:]
    if not np.isnan(leftarm).all():
      amigos_dataset.append(extract_row(leftarm, amigos_frequency, valence, arousal))
    if not np.isnan(rightarm).all():  
      amigos_dataset.append(extract_row(rightarm, amigos_frequency, valence, arousal))
len(amigos_dataset)

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 197, using nperseg = 197
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 197, using nperseg = 197
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 197, using nperseg = 197
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 197, using nperseg = 197
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 195, using nperseg = 195
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 191, using nperseg = 191
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 83, using nperseg = 83
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 106, using nperseg = 106
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 81, using nperseg = 81
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 107, using nperseg = 107
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 145, using nperseg = 145
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\s

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 96, using nperseg = 96
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 72, using nperseg = 72
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 133, using nperseg = 133
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 196, using nperseg = 196
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 196, using nperseg = 196
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\s

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 145, using nperseg = 145
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 156, using nperseg = 156
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 95, using nperseg = 95
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 99, using nperseg = 99
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 133, using nperseg = 133
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\s

...

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 189, using nperseg = 189
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 185, using nperseg = 185
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 187, using nperseg = 187
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 187, using nperseg = 187
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 193, using nperseg = 193
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 84, using nperseg = 84
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 198, using nperseg = 198
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 194, using nperseg = 194
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 198, using nperseg = 198
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 198, using nperseg = 198
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 184, using nperseg = 184
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 184, using nperseg = 184
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 163, using nperseg = 163
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 163, using nperseg = 163
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 106, using nperseg = 106
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packag

D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 187, using nperseg = 187
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 109, using nperseg = 109
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 73, using nperseg = 73
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 137, using nperseg = 137
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 141, using nperseg = 141
  .format(nperseg, input_length))
D:\Users\Kamil\Anaconda3\lib\site-packages

1258

## Merge datasets and convert to dataframe

In [18]:
dataset = np.concatenate((ascertain_dataset, deap_dataset, decaf_dataset, amigos_dataset))
dataframe = pd.DataFrame(columns=['mean_hr', 'min_hr', 'max_hr', 'std_hr', 'kurtosis_hr', 'skewness_hr', 'hr_above_mean_plus_std', 'hr_below_mean_minus_std', 'mad_rr', 'mean_rr', 'kurtosis_rr', 'skewness_rr', 'rr_above_mean_plus_std', 'rr_below_mean_minus_std', 'sdnn', 'rmssd', 'sdsd', 'pnn20', 'pnn50', 'sd1', 'sd2', 'sd2_sd1_ratio', 'lf', 'hf', 'lfhf', 'valence', 'arousal'])
for i in range(len(dataset)):
  d = dataset[i]
  dataframe.loc[i] = [d['mean_hr'],  d['min_hr'], d['max_hr'], d['std_hr'], d['kurtosis_hr'], d['skewness_hr'], d['hr_above_mean_plus_std'], d['hr_below_mean_minus_std'], d['mad_rr'], d['mean_rr'], d['kurtosis_rr'], d['skewness_rr'], d['rr_above_mean_plus_std'], d['rr_below_mean_minus_std'], d['sdnn'], d['rmssd'], d['sdsd'], d['pnn20'], d['pnn50'], d['sd1'], d['sd2'], d['sd2_sd1_ratio'], d['lf'], d['hf'], d['lfhf'], d['valence'], d['arousal']]
dataframe = dataframe.dropna()
dataframe

,mean_hr,min_hr,max_hr,std_hr,kurtosis_hr,skewness_hr,hr_above_mean_plus_std,hr_below_mean_minus_std,mad_rr,mean_rr,...,pnn20,pnn50,sd1,sd2,sd2_sd1_ratio,lf,hf,lfhf,valence,arousal
0,74.583030,67.665198,93.090909,5.305757,2.991777,1.781948,0.118644,0.050847,0.034748,0.808130,...,0.593220,0.169492,0.018971,0.070505,3.716455,211.577771,395.430415,0.535057,6.333333,2.333333
1,74.596844,67.665198,93.090909,5.316827,2.959597,1.766687,0.118644,0.050847,0.034748,0.807998,...,0.644068,0.186441,0.018026,0.070955,3.936358,210.874262,418.965946,0.503321,6.333333,2.333333
2,73.926031,69.189189,79.585492,2.525467,-0.883386,0.233715,0.196721,0.163934,0.034748,0.812564,...,0.672131,0.180328,0.011656,0.037236,3.194415,58.345201,677.128418,0.086166,7.666667,5.000000
3,73.932655,68.878924,80.000000,2.537906,-0.762132,0.248377,0.196721,0.147541,0.034748,0.812500,...,0.704918,0.147541,0.011588,0.037438,3.230628,57.738214,673.401980,0.085741,7.666667,5.000000
4,76.810345,71.441860,85.810056,3.278622,-0.315709,0.372148,0.158730,0.190476,0.034748,0.782552,...,0.634921,0.158730,0.011682,0.045204,3.869581,83.332183,474.298575,0.175696,2.333333,2.333333
5,76.813624,71.111111,85.333333,3.311730,-0.463103,0.286283,0.158730,0.174603,0.034748,0.782552,...,0.634921,0.111111,0.011820,0.045817,3.876070,87.907807,517.797088,0.169773,2.333333,2.333333
6,90.719195,72.452830,101.052632,7.738055,-0.558265,-0.739916,0.162162,0.202703,0.046331,0.666596,...,0.513514,0.094595,0.012873,0.085815,6.666288,188.347918,220.714943,0.853354,5.000000,9.000000
7,90.711908,72.452830,101.721854,7.745401,-0.588949,-0.714934,0.148649,0.189189,0.046331,0.666649,...,0.540541,0.094595,0.012652,0.085751,6.777737,186.989126,226.327854,0.826187,5.000000,9.000000
8,70.615766,66.206897,77.969543,2.696767,-0.142372,0.500768,0.189655,0.137931,0.034748,0.850889,...,0.637931,0.224138,0.014565,0.042834,2.940898,183.225111,556.470865,0.329263,6.333333,6.333333
9,70.768317,66.493506,77.575758,2.618493,-0.256425,0.474164,0.214286,0.196429,0.034748,0.848982,...,0.660714,0.196429,0.014377,0.041361,2.876855,107.630278,447.785828,0.240361,6.333333,6.333333


## Create dataframe for classification + train/test split

In [19]:
def get_class_by_both(valence, arousal):
    if valence < 3.66:
        if arousal < 3.66:
            return "vL_aL"
        if 3.66 <= arousal < 6.33:
            return "vL_aM"
        return "vL_aH"
    elif 3.66 <= valence < 6.33:
        if arousal < 3.66:
            return "vM_aL"
        if 3.66 <= arousal < 6.33:
            return "vM_aM"
        return "vM_aH"
    else:
        if arousal < 3.66:
            return "vH_aL"
        if 3.66 <= arousal < 6.33:
            return "vH_aM"
        return "vH_aH"

# dataframe_cls[['emotion_class']].groupby(by='emotion_class').size()
dataframe_cls = dataframe.copy()
valences = dataframe_cls[['valence']].to_numpy()
arousals = dataframe_cls[['arousal']].to_numpy()
classes = []
for i in range(0,len(valences)):
  valence = valences[i]
  arousal = arousals[i]
  classes.append(get_class_by_both(valence,arousal))

dataframe_cls['emotion_class'] = classes
dataframe_cls = dataframe_cls.drop(['valence', 'arousal'], axis=1)
dataframe_cls[['emotion_class']].groupby(by='emotion_class').size()

emotion_class
vH_aH    1813
vH_aL     491
vH_aM     927
vL_aH    1030
vL_aL     240
vL_aM     574
vM_aH     843
vM_aL     529
vM_aM    1055
dtype: int64

## Dump dataframes to csv files

In [3]:
dataframe.to_csv('D:\Physiological Data\dataframe.csv', index=False)
dataframe_cls.to_csv('D:\Physiological Data\dataframe_cls.csv', index=False)

NameError: name 'dataframe' is not defined

# Learning
## Load dataframes

In [24]:
dataframe = pd.read_csv('D:\Physiological Data\dataframe.csv')
dataframe_cls = pd.read_csv('D:\Physiological Data\dataframe_cls.csv')

## Train test split of dataframes

In [32]:
train_set, test_set = train_test_split(dataframe, test_size=0.2)
train, validation = train_test_split(train_set, test_size=0.2)

X_train = train_set.drop(['valence', 'arousal'], axis=1)
#X_train = train.drop(['valence', 'arousal', 'hr_start', 'hr_max_', 'hr_min_', 'hr_mean', 'hr_median','hr_at3', 'hr_at6', 'hr_at9', 'gsr_start', 'gsr_max_', 'gsr_min_', 'gsr_mean', 'gsr_median', 'gsr_at3', 'gsr_at6', 'gsr_at9'], axis=1)
Y_train = train_set[['valence', 'arousal']]
X_test = test_set.drop(['valence', 'arousal'], axis=1)
#X_train = train.drop(['valence', 'arousal', 'hr_start', 'hr_max_', 'hr_min_', 'hr_mean', 'hr_median','hr_at3', 'hr_at6', 'hr_at9', 'gsr_start', 'gsr_max_', 'gsr_min_', 'gsr_mean', 'gsr_median', 'gsr_at3', 'gsr_at6', 'gsr_at9'], axis=1)
Y_test = test_set[['valence', 'arousal']]

train_set_cls, test_set_cls = train_test_split(dataframe_cls, test_size=0.2)
train, validation = train_test_split(train_set_cls, test_size=0.2)

X_train_cls = train_set_cls.drop(['emotion_class'], axis=1)
Y_train_cls = train_set_cls[['emotion_class']]
X_test_cls = test_set_cls.drop(['emotion_class'], axis=1)
Y_test_cls = test_set_cls[['emotion_class']]

## Multioutput random forest regression
Results - really low, not getting good results from this (0.1 R^2 score)

In [26]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

kfold = KFold(n_splits=10)

regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, max_depth=20, random_state=0))
results = cross_val_score(regr_multirf,X_train_scaled, Y_train, cv=kfold)
print(results.mean())

regr_multirf = MultiOutputRegressor(DummyRegressor())
results = cross_val_score(regr_multirf,X_train_scaled, Y_train, cv=kfold)
print(results.mean())

0.06278396397593133
-0.0017712838687260813


## Extra trees regression with hyperopt parameter search
Results - also really low, better than multioutput but still low

In [6]:
extra_estim = HyperoptEstimator(regressor=extra_trees_regression('my_regre'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=300,
                          trial_timeout=60)

extra_estim.fit(X_train.to_numpy(), Y_train.to_numpy())

extra_best = extra_estim.best_model()['learner']
print(extra_estim.best_model())
extra_scaled = X_test
if (len(extra_estim.best_model()['preprocs']) > 0):
  extra_prep = extra_estim.best_model()['preprocs'][0]
  extra_scaled = extra_prep.transform(X_test)
print(extra_best.score(extra_scaled, Y_test))

100%|█████████████████████████████████████████████████████| 1/1 [00:42<00:00, 42.43s/it, best loss: 0.8843349218654841]


100%|█████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.95s/it, best loss: 0.8838260266202713]


100%|█████████████████████████████████████████████████████| 1/1 [00:04<00:00,  5.00s/it, best loss: 0.8838260266202713]
{'learner': ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features=0.4994783162129297, max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=463, n_jobs=1,
          oob_score=False, random_state=1, verbose=False, warm_start=False), 'preprocs': (MinMaxScaler(copy=True, feature_range=(-1.0, 1.0)),), 'ex_preprocs': ()}
0.12147796306977585


## Hyperopt estimator for random forest regression (multi output)
Similar to previous ones

In [7]:
regression_estim = HyperoptEstimator(regressor=random_forest_regression('my_regre'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=300,
                          trial_timeout=60)

regression_estim.fit(X_train.to_numpy(), Y_train.to_numpy())

random_best = regression_estim.best_model()['learner']
print(regression_estim.best_model())
random_scaled = X_test
if (len(regression_estim.best_model()['preprocs']) > 0):
  ranom_prep = regression_estim.best_model()['preprocs'][0]
  random_scaled = ranom_prep.transform(X_test)
print(random_best.score(random_scaled, Y_test))

100%|█████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.27s/it, best loss: 0.8965749365146981]


100%|█████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.34s/it, best loss: 0.8928839322218622]


100%|█████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.08s/it, best loss: 0.8928839322218622]
{'learner': RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features=0.0016876586521818587, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=373, n_jobs=1,
           oob_score=False, random_state=3, verbose=False,
           warm_start=False), 'preprocs': (MinMaxScaler(copy=True, feature_range=(0.0, 1.0)),), 'ex_preprocs': ()}
0.11050749424726501


## Hyperopt estimator for gradient boosting (separate for valence and arousal)
Only to check, we see that regression does not give good results here

In [8]:
valence_gradient_estim = HyperoptEstimator(regressor=gradient_boosting_regression('my_regre'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=300,
                          trial_timeout=60)

valence_gradient_estim.fit(X_train.to_numpy(), Y_train[['valence']].to_numpy().ravel())

valence_gradient_best = valence_gradient_estim.best_model()['learner']
print(valence_gradient_estim.best_model())
valence_gradient_scaled = X_test
if (len(valence_gradient_estim.best_model()['preprocs']) > 0):
  valence_gradient_prep = valence_gradient_estim.best_model()['preprocs'][0]
  valence_gradient_scaled = valence_gradient_prep.transform(X_test)
print(valence_gradient_best.score(valence_gradient_scaled, Y_test[['valence']]))

arousal_gradient_estim = HyperoptEstimator(regressor=gradient_boosting_regression('my_regre'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=300,
                          trial_timeout=60)

arousal_gradient_estim.fit(X_train.to_numpy(), Y_train[['arousal']].to_numpy().ravel())

arousal_gradient_best = arousal_gradient_estim.best_model()['learner']
print(arousal_gradient_estim.best_model())
arousal_gradient_scaled = X_test
if (len(arousal_gradient_estim.best_model()['preprocs']) > 0):
  arousal_gradient_prep = arousal_gradient_estim.best_model()['preprocs'][0]
  arousal_gradient_scaled = arousal_gradient_prep.transform(X_test)
print(arousal_gradient_best.score(arousal_gradient_scaled, Y_test[['arousal']]))

100%|█████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.28s/it, best loss: 0.9108253396069945]


100%|█████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.02s/it, best loss: 0.9108253396069945]


100%|█████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.02s/it, best loss: 0.9108253396069945]
{'learner': GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05106002108192642, loss='lad', max_depth=None,
             max_features=0.8839598548193086, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=3, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=213,
             n_iter_no_change=None, presort='auto', random_state=0,
             subsample=0.9963298582213607, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False), 'preprocs': (StandardScaler(copy=True, with_mean=False, with_std=True),), 'ex_preprocs': ()}
0.06015591194876901
  0%|                                                                              | 0/1 [00:00<?, ?it/s, best loss: ?]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

False

## Hyperopt estimator for SGD (separate for valence and arousal)
Only to check, we see that regression does not give good results here

In [10]:
valence_sgd_estim = HyperoptEstimator(regressor=sgd_regression('my_regre'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=300,
                          trial_timeout=60)

valence_sgd_estim.fit(X_train.to_numpy(), Y_train[['valence']].to_numpy().ravel())

valence_sgd_best = valence_sgd_estim.best_model()['learner']
print(valence_sgd_estim.best_model())
valence_sgd_scaled = X_test
if(len(valence_sgd_estim.best_model()['preprocs']) > 0):
  valence_sgd_prep = valence_sgd_estim.best_model()['preprocs'][0]
  valence_sgd_scaled = valence_sgd_prep.transform(X_test)
print(valence_sgd_best.score(valence_sgd_scaled, Y_test[['valence']]))

arousal_sgd_estim = HyperoptEstimator(regressor=sgd_regression('my_regre'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=300,
                          trial_timeout=60)

arousal_sgd_estim.fit(X_train.to_numpy(), Y_train[['arousal']].to_numpy().ravel())

arousal_sgd_best = arousal_sgd_estim.best_model()['learner']
print(arousal_sgd_estim.best_model())
arousal_sgd_scaled = X_test
if (len(arousal_sgd_estim.best_model()['preprocs']) > 0):
  arousal_sgd_prep = arousal_sgd_estim.best_model()['preprocs'][0]
  arousal_sgd_scaled = arousal_sgd_prep.transform(X_test)
print(arousal_sgd_best.score(arousal_sgd_scaled, Y_test[['arousal']]))

100%|█████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.40s/it, best loss: 0.9940440416840398]


100%|█████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.96s/it, best loss: 0.9899151123884333]


100%|█████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.62s/it, best loss: 0.9899151123884333]
{'learner': SGDRegressor(alpha=0.01645728377884476, average=False, early_stopping=False,
       epsilon=0.02331105924864922, eta0=0.024693813018983365,
       fit_intercept=True, l1_ratio=0.41976184207536915,
       learning_rate='invscaling', loss='squared_loss',
       max_iter=14778823.0, n_iter=None, n_iter_no_change=5, penalty='l2',
       power_t=0.6749243082344943, random_state=2, shuffle=True,
       tol=0.00016611018034564775, validation_fraction=0.1, verbose=False,
       warm_start=False), 'preprocs': (StandardScaler(copy=True, with_mean=True, with_std=True),), 'ex_preprocs': ()}
-0.001811853122442919
100%|█████████████████████████████████████████████████████| 1/1 [00:04<00:00,  5.00s/it, best loss: 0.9984326841529948]


100%|█████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.44s/it, best loss: 0.9900208431418156]


100%|█████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.36s/it, best loss: 0.9900208431418156]
{'learner': SGDRegressor(alpha=0.0010693167974656869, average=False, early_stopping=False,
       epsilon=0.0018378527224496666, eta0=0.001438597643681156,
       fit_intercept=True, l1_ratio=0.1619875777438354,
       learning_rate='constant', loss='squared_epsilon_insensitive',
       max_iter=24207588.0, n_iter=None, n_iter_no_change=5, penalty='l1',
       power_t=0.08498722720176514, random_state=4, shuffle=True,
       tol=0.003909687644378499, validation_fraction=0.1, verbose=False,
       warm_start=False), 'preprocs': (MinMaxScaler(copy=True, feature_range=(-1.0, 1.0)),), 'ex_preprocs': ()}
0.0024460934730676787


## Hyperopt estimator for any regressor (separate for valence and arousal)
Only to check, we see that regression does not give good results here

In [13]:
any_reg_valence_estim = HyperoptEstimator(regressor=any_regressor('my_regre'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=300,
                          trial_timeout=60)

any_reg_valence_estim.fit(X_train.to_numpy(), Y_train[['valence']].to_numpy().ravel())

any_reg_valence_best = any_reg_valence_estim.best_model()['learner']
print(any_reg_valence_estim.best_model())
any_reg_valence_scaled = X_test
if (len(any_reg_valence_estim.best_model()['preprocs']) > 0):
  any_reg_valence_prep = any_reg_valence_estim.best_model()['preprocs'][0]
  any_reg_valence_scaled = any_reg_valence_prep.transform(X_test)
print(any_reg_valence_best.score(any_reg_valence_scaled, Y_test[['valence']]))

any_reg_arousal_estim = HyperoptEstimator(regressor=any_regressor('my_regre'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=300,
                          trial_timeout=60)

any_reg_arousal_estim.fit(X_train.to_numpy(), Y_train[['arousal']].to_numpy().ravel())

any_reg_arousal_best = any_reg_arousal_estim.best_model()['learner']
print(any_reg_arousal_estim.best_model())
any_reg_arousal_scaled = X_test
if (len(any_reg_arousal_estim.best_model()['preprocs']) > 0):
  any_reg_arousal_prep = any_reg_arousal_estim.best_model()['preprocs'][0]
  any_reg_arousal_scaled = any_reg_arousal_prep.transform(X_test)
print(any_reg_arousal_best.score(any_reg_arousal_scaled, Y_test[['arousal']]))

100%|█████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.61s/it, best loss: 0.8906761813024171]


100%|█████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.94s/it, best loss: 0.8904693049843515]


100%|█████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.96s/it, best loss: 0.8904693049843515]
{'learner': ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='log2', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=1728, n_jobs=1,
          oob_score=False, random_state=2, verbose=False, warm_start=False), 'preprocs': (MinMaxScaler(copy=True, feature_range=(-1.0, 1.0)),), 'ex_preprocs': ()}
0.12409335677757916
100%|█████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.12s/it, best loss: 0.8984335321035603]


  0%|                                                                              | 0/1 [00:00<?, ?it/s, best loss: ?]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## Classification (any classifier with hyperopt)
Really good results, mean accuracy around 0.35-0.37 for 9 classes, also starting to dump estimators here

In [14]:
estim = HyperoptEstimator(classifier=any_classifier('my_clf_any'),
                          preprocessing=any_preprocessing('my_pre_any'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=60)

estim.fit(X_train_cls.to_numpy(), Y_train_cls.to_numpy().ravel())
m = estim.best_model()['learner']
print(estim.best_model())
scaled = X_test_cls
if (len(estim.best_model()['preprocs']) > 0):
  prep = estim.best_model()['preprocs'][0]
  dump(prep, "D:\Physiological Data\prep.joblib")
  scaled = prep.transform(X_test_cls)
print(m.score(scaled, Y_test_cls))

dump(m, "D:\Physiological Data\\best_model.joblib")
dump(estim, "D:\Physiological Data\estim.joblib")


100%|█████████████████████████████████████████████████████| 1/1 [00:31<00:00, 31.14s/it, best loss: 0.6885928393005829]
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth=None, max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=521, n_jobs=1,
           oob_score=False, random_state=2, verbose=False,
           warm_start=False), 'preprocs': (StandardScaler(copy=True, with_mean=True, with_std=True),), 'ex_preprocs': ()}
0.3570952698201199


['D:\\Physiological Data\\estim.joblib']

## Continue from saved hyperopt estimator (this is for keeping checkpoints)

In [15]:
estimator = load("D:\Physiological Data\estim.joblib")
iterations = 6
for i in range(0,iterations):
  estimator.fit(X_train_cls.to_numpy(), Y_train_cls.to_numpy().ravel(), warm_start=True)
  m = estimator.best_model()['learner']
  print(estimator.best_model())
  scaled = X_test_cls
  if (len(estimator.best_model()['preprocs']) > 0):
    prep = estimator.best_model()['preprocs'][0]
    dump(prep, "D:\Physiological Data\prep.joblib")
    scaled = prep.transform(X_test_cls)
  print(m.score(scaled, Y_test_cls))
  dump(m, "D:\Physiological Data\\best_model.joblib")
  dump(estimator, "D:\Physiological Data\estim.joblib")

100%|█████████████████████████████████████████████████████| 1/1 [00:31<00:00, 31.88s/it, best loss: 0.6819317235636969]
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features=0.504230335098639,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=359, n_jobs=1, oob_score=False, random_state=2,
           verbose=False, warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
0.351765489673551
100%|█████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.44s/it, best loss: 0.6819317235636969]


100%|█████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.46s/it, best loss: 0.6802664446294755]
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=27, n_jobs=1,
           oob_score=False, random_state=3, verbose=False,


           warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
0.3384410393071286
100%|█████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.42s/it, best loss: 0.6794338051623647]
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=34, n_jobs=1,
           oob_score=False, random_state=3, verbose=False,
           warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
0.346435709526982
100%|█████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.59s/it, best loss: 0.6744379683597003]


100%|█████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.93s/it, best loss: 0.6727726894254787]
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=56, n_jobs=1,
           oob_score=False, random_state=3, verbose=False,
           warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
0.34377081945369753
100%|█████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.95s/it, best loss: 0.6727726894254787]


100%|█████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.09s/it, best loss: 0.6727726894254787]
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=56, n_jobs=1,
           oob_score=False, random_state=3, verbose=False,
           warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
0.34377081945369753
100%|█████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.26s/it, best loss: 0.6727726894254787]


100%|█████████████████████████████████████████████████████| 1/1 [00:55<00:00, 55.71s/it, best loss: 0.6727726894254787]
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=56, n_jobs=1,
           oob_score=False, random_state=3, verbose=False,
           warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
0.34377081945369753


## Trying to use hyperopt with the best classifier (random forest or extra trees were the best) to optimize classifier parameter
Results like before, around 0.37 accuracy
Not saving estimator, just model and scaler

In [ ]:
from hpsklearn import extra_trees, random_forest
estim_2 = HyperoptEstimator(classifier=extra_trees('my_clf_ext'),
                          preprocessing=any_preprocessing('my_pre_clf_ext'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=60)

estim_2.fit(X_train_cls.to_numpy(), Y_train_cls.to_numpy().ravel())
m_2 = estim_2.best_model()['learner']
print(estim_2.best_model())
scaled_2 = X_test_cls
if (len(estim_2.best_model()['preprocs']) > 0):
  prep_2 = estim_2.best_model()['preprocs'][0]
  dump(prep_2, "D:\Physiological Data\prep2.joblib")
  scaled_2 = prep_2.transform(X_test_cls)
print(m_2.score(scaled_2, Y_test_cls))
dump(m_2, "D:\Physiological Data\\best_model2.joblib")

iters = 10
for i in range(0,iters):
  estim_2.fit(X_train_cls.to_numpy(), Y_train_cls.to_numpy().ravel(), warm_start=True)
  m_2 = estim_2.best_model()['learner']
  print(estim_2.best_model())
  scaled_2 = X_test_cls
  if (len(estim_2.best_model()['preprocs']) > 0):
    prep_2 = estim_2.best_model()['preprocs'][0]
    dump(prep_2, "D:\Physiological Data\prep2.joblib")
    scaled_2 = prep_2.transform(X_test_cls)
  print(m_2.score(scaled_2, Y_test_cls))
  dump(m_2, "D:\Physiological Data\\best_model2.joblib")

100%|█████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.97s/it, best loss: 0.6477935054121565]
{'learner': ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1178, n_jobs=1,
           oob_score=False, random_state=2, verbose=False,
           warm_start=False), 'preprocs': (MinMaxScaler(copy=True, feature_range=(0.0, 1.0)),), 'ex_preprocs': ()}
0.37308461025982675
100%|█████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.73s/it, best loss: 0.6477935054121565]


100%|█████████████████████████████████████████████████████| 1/1 [00:27<00:00, 27.82s/it, best loss: 0.6477935054121565]
{'learner': ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1178, n_jobs=1,
           oob_score=False, random_state=2, verbose=False,
           warm_start=False), 'preprocs': (MinMaxScaler(copy=True, feature_range=(0.0, 1.0)),), 'ex_preprocs': ()}
0.37308461025982675
100%|█████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.86s/it, best loss: 0.6477935054121565]


100%|█████████████████████████████████████████████████████| 1/1 [00:21<00:00, 21.78s/it, best loss: 0.6477935054121565]
{'learner': ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1178, n_jobs=1,
           oob_score=False, random_state=2, verbose=False,
           warm_start=False), 'preprocs': (MinMaxScaler(copy=True, feature_range=(0.0, 1.0)),), 'ex_preprocs': ()}
0.37308461025982675
100%|█████████████████████████████████████████████████████| 1/1 [00:33<00:00, 33.56s/it, best loss: 0.6477935054121565]


100%|█████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.42s/it, best loss: 0.6477935054121565]
{'learner': ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
           max_depth=None, max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1178, n_jobs=1,
           oob_score=False, random_state=2, verbose=False,
           warm_start=False), 'preprocs': (MinMaxScaler(copy=True, feature_range=(0.0, 1.0)),), 'ex_preprocs': ()}
0.37308461025982675
  0%|                                                                              | 0/1 [00:00<?, ?it/s, best loss: ?]

## Statistics calculation

In [33]:
from sklearn import metrics
def scores_for_model(model, X_train, Y_train, X_test, Y_test):
  estimat = HyperoptEstimator(classifier=model,
                          preprocessing=any_preprocessing('my_pre_clf_ext'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=300)
  estimat.fit(X_train.to_numpy(), Y_train.to_numpy().ravel())
  model = estimat.best_model()['learner']
  print(estimat.best_model())
  scaled_x = X_test
  if (len(estimat.best_model()['preprocs']) > 0):
    scaler = estimat.best_model()['preprocs'][0]
    scaled_x = scaler.transform(X_test)
  predicted = model.predict(scaled_x)
  print("Accuracy: " + str(metrics.accuracy_score(Y_test, predicted)))
  print("Recall:" + str(metrics.recall_score(Y_test, predicted, average='micro')))
  print("Precision: " + str(metrics.precision_score(Y_test, predicted, average='micro')))
  print("F1: " + str(metrics.f1_score(Y_test, predicted, average='micro')))
  print(metrics.precision_recall_fscore_support(Y_test_cls, predicted))
  
def try_scores_for_model(model, X_train, Y_train, X_test, Y_test):
  try:
    scores_for_model(model, X_train, Y_train, X_test, Y_test)
  except:
    try_scores_for_model(model, X_train, Y_train, X_test, Y_test)

models = [extra_trees('ext'), random_forest('rand'), 
#           knn('knn'), 
          svc('svc')
#           sgd('sgd'), ada_boost('ada'), gradient_boosting('grad'), multinomial_nb('multnb'), gaussian_nb('gaussiannb')
         ]
extra = ExtraTreesClassifier()
randomf = RandomForestClassifier()
svmc = SVC()
gradientBoosting = GradientBoostingClassifier()
knc = KNeighborsClassifier()

scal = preprocessing.StandardScaler().fit(X_train_cls)
X_train_scal = scal.transform(X_train_cls)
X_test_scal = scal.transform(X_test_cls)

extra.fit(X_train_scal, Y_train_cls.to_numpy())
randomf.fit(X_train_scal, Y_train_cls.to_numpy())
svmc.fit(X_train_scal, Y_train_cls.to_numpy())
gradientBoosting.fit(X_train_scal, Y_train_cls.to_numpy())
knc.fit(X_train_scal, Y_train_cls.to_numpy())


predicted = extra.predict(X_test_scal)
print(extra)
print("Accuracy: " + str(metrics.accuracy_score(Y_test_cls, predicted)))
print("Recall:" + str(metrics.recall_score(Y_test_cls, predicted, average='micro')))
print("Precision: " + str(metrics.precision_score(Y_test_cls, predicted, average='micro')))
print("F1: " + str(metrics.f1_score(Y_test_cls, predicted, average='micro')))
scores = cross_val_score(extra, X_train_cls, Y_train_cls, cv=5)
print("Cross validated: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(metrics.precision_recall_fscore_support(Y_test_cls, predicted))

predicted = randomf.predict(X_test_scal)
print(randomf)
print("Accuracy: " + str(metrics.accuracy_score(Y_test_cls, predicted)))
print("Recall:" + str(metrics.recall_score(Y_test_cls, predicted, average='micro')))
print("Precision: " + str(metrics.precision_score(Y_test_cls, predicted, average='micro')))
print("F1: " + str(metrics.f1_score(Y_test_cls, predicted, average='micro')))
scores = cross_val_score(randomf, X_train_cls, Y_train_cls, cv=5)
print("Cross validated: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(metrics.precision_recall_fscore_support(Y_test_cls, predicted))

predicted = svmc.predict(X_test_scal)
print(svmc)
print("Accuracy: " + str(metrics.accuracy_score(Y_test_cls, predicted)))
print("Recall:" + str(metrics.recall_score(Y_test_cls, predicted, average='micro')))
print("Precision: " + str(metrics.precision_score(Y_test_cls, predicted, average='micro')))
print("F1: " + str(metrics.f1_score(Y_test_cls, predicted, average='micro')))
scores = cross_val_score(svmc, X_train_cls, Y_train_cls, cv=5)
print("Cross validated: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(metrics.precision_recall_fscore_support(Y_test_cls, predicted))

predicted = gradientBoosting.predict(X_test_scal)
print(gradientBoosting)
print("Accuracy: " + str(metrics.accuracy_score(Y_test_cls, predicted)))
print("Recall:" + str(metrics.recall_score(Y_test_cls, predicted, average='micro')))
print("Precision: " + str(metrics.precision_score(Y_test_cls, predicted, average='micro')))
print("F1: " + str(metrics.f1_score(Y_test_cls, predicted, average='micro')))
scores = cross_val_score(svmc, X_train_cls, Y_train_cls, cv=5)
print("Cross validated: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(metrics.precision_recall_fscore_support(Y_test_cls, predicted))

predicted = knc.predict(X_test_scal)
print(knc)
print("Accuracy: " + str(metrics.accuracy_score(Y_test_cls, predicted)))
print("Recall:" + str(metrics.recall_score(Y_test_cls, predicted, average='micro')))
print("Precision: " + str(metrics.precision_score(Y_test_cls, predicted, average='micro')))
print("F1: " + str(metrics.f1_score(Y_test_cls, predicted, average='micro')))
scores = cross_val_score(svmc, X_train_cls, Y_train_cls, cv=5)
print("Cross validated: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(metrics.precision_recall_fscore_support(Y_test_cls, predicted))
# for mod in models:
#   scores_for_model(mod, X_train_cls, Y_train_cls, X_test_cls, Y_test_cls)

C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\Kamil\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\Kamil\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-pa

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
Accuracy: 0.31245836109260494
Recall:0.31245836109260494
Precision: 0.31245836109260494
F1: 0.31245836109260494


C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\model_s

Cross validated: 0.29 (+/- 0.03)
(array([0.34533552, 0.40677966, 0.23529412, 0.29032258, 0.2       ,
       0.30864198, 0.29059829, 0.21875   , 0.33333333]), array([0.56266667, 0.22018349, 0.21390374, 0.31188119, 0.08333333,
       0.22522523, 0.22368421, 0.14141414, 0.24770642]), array([0.42799189, 0.28571429, 0.22408964, 0.30071599, 0.11764706,
       0.26041667, 0.2527881 , 0.17177914, 0.28421053]), array([375, 109, 187, 202,  48, 111, 152,  99, 218], dtype=int64))
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy: 0.2858094603597602
Recall:0.2858094603597602
Precision: 0.2858094603597602
F1: 0.285809460359

C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\model_s

...

(array([0.25716332, 0.        , 0.11764706, 0.3       , 0.        ,
       0.        , 0.        , 0.        , 0.26388889]), array([0.95733333, 0.        , 0.01069519, 0.01485149, 0.        ,
       0.        , 0.        , 0.        , 0.08715596]), array([0.40542067, 0.        , 0.01960784, 0.02830189, 0.        ,
       0.        , 0.        , 0.        , 0.13103448]), array([375, 109, 187, 202,  48, 111, 152,  99, 218], dtype=int64))
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
Accuracy: 0.243

C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\base.py:196: FutureWarning: The defa

Cross validated: 0.25 (+/- 0.01)
(array([0.26156752, 0.125     , 0.23076923, 0.20909091, 0.07142857,
       0.1       , 0.16923077, 0.14814815, 0.2578125 ]), array([0.73866667, 0.01834862, 0.06417112, 0.11386139, 0.02083333,
       0.02702703, 0.07236842, 0.04040404, 0.15137615]), array([0.38633194, 0.032     , 0.10041841, 0.1474359 , 0.03225806,
       0.04255319, 0.10138249, 0.06349206, 0.19075145]), array([375, 109, 187, 202,  48, 111, 152,  99, 218], dtype=int64))
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, 

C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\base.py:196: FutureWarning: The defa

Cross validated: 0.25 (+/- 0.01)
(array([0.27687776, 0.18390805, 0.15254237, 0.18709677, 0.22222222,
       0.13559322, 0.1496063 , 0.05      , 0.21621622]), array([0.50133333, 0.14678899, 0.14438503, 0.14356436, 0.04166667,
       0.07207207, 0.125     , 0.03030303, 0.14678899]), array([0.35673624, 0.16326531, 0.14835165, 0.16246499, 0.07017544,
       0.09411765, 0.13620072, 0.03773585, 0.17486339]), array([375, 109, 187, 202,  48, 111, 152,  99, 218], dtype=int64))


In [6]:
scores_for_model(extra_trees('ext'), X_train_cls, Y_train_cls, X_test_cls, Y_test_cls)

100%|█████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.47s/it, best loss: 0.6669442131557035]
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth=None, max_features=0.027423386031542717,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1183, n_jobs=1, oob_score=False, random_state=3,
           verbose=False, warm_start=False), 'preprocs': (StandardScaler(copy=True, with_mean=False, with_std=True),), 'ex_preprocs': ()}
Accuracy: 0.3637574950033311
Recall:0.3637574950033311
Precision: 0.3637574950033311
F1: 0.36375749500333104


In [36]:
scores_for_model(random_forest('rand'), X_train_cls, Y_train_cls, X_test_cls, Y_test_cls)

100%|█████████████████████████████████████████████████████| 1/1 [00:50<00:00, 50.41s/it, best loss: 0.6969192339716903]


100%|████████████████████████████████████████████████████| 1/1 [01:42<00:00, 102.66s/it, best loss: 0.6919233971690257]
{'learner': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.003783055594982085,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=467, n_jobs=1, oob_score=False, random_state=4,
            verbose=False, warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
Accuracy: 0.357761492338441
Recall:0.357761492338441
Precision: 0.357761492338441
F1: 0.357761492338441
(array([0.32862191, 0.70967742, 0.35      , 0.39520958, 0.57142857,
       0.51282051, 0.3956044 , 0.23255814, 0.37356322]), array([0.744     , 0.20183486, 0.18716578, 0.32673267, 0.08333333,
       0.18018018, 0.23684211, 0.1010101 , 0.29816514]), array([0.45588235, 0.31428571, 0.24390244,

In [34]:
scores_for_model(svc('svc'), X_train_cls, Y_train_cls, X_test_cls, Y_test_cls)

100%|█████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.19s/it, best loss: 0.7252289758534555]


100%|█████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.03s/it, best loss: 0.7252289758534555]
{'learner': SVC(C=0.860955680092458, cache_size=512, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=1, gamma=119.77120966667245,
  kernel='rbf', max_iter=293156245.0, probability=False, random_state=3,
  shrinking=False, tol=2.302875997333159e-05, verbose=False), 'preprocs': (PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True),), 'ex_preprocs': ()}
Accuracy: 0.28914057295136575
Recall:0.28914057295136575
Precision: 0.28914057295136575
F1: 0.28914057295136575


C:\Users\Kamil\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.26349206, 0.8       , 0.42105263, 0.42465753, 0.        ,
       0.72727273, 0.36585366, 0.5       , 0.37681159]), array([0.88533333, 0.03669725, 0.0855615 , 0.15346535, 0.        ,
       0.07207207, 0.09868421, 0.02020202, 0.11926606]), array([0.40611621, 0.07017544, 0.14222222, 0.22545455, 0.        ,
       0.13114754, 0.15544041, 0.03883495, 0.18118467]), array([375, 109, 187, 202,  48, 111, 152,  99, 218], dtype=int64))
